# Imports and utility functions

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd
import re

In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

# Load data

In [ ]:
from sklearn.utils import shuffle

dataset = pd.read_csv('train_data_imdb.csv')
dataset = shuffle(dataset)
dataset.tail()

,Unnamed: 0,text,label
12341,23856,Why would any legitimate actor having read the...,0
17921,6432,"And my children love it now! Granted, I can wa...",1
1233,4408,Using Buster Keaton in the twilight of his car...,1
13700,14169,"Minimal script, minimal character development,...",0
10062,8556,"Not really a big box office draw, but I was pl...",1


In [ ]:
x_train = list(cleanTexts(dataset['text']))
# print(x[:5])

y_train = list(dataset['label'])
# print(y[:5])

#x_train = x_train[:10000]
#y_train = y_train[:10000]

x_train = x_train[:25000]
y_train = y_train[:25000]

In [ ]:
test_data = pd.read_csv('test_data_imdb.csv')

x_test = list(cleanTexts(test_data['text']))
y_test = list(test_data['label'])

x_test = x_test[:10000]
y_test = y_test[:10000]

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

# Import FastText embeddings

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz

--2022-04-17 10:54:43--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  17.2MB/s    in 69s     

2022-04-17 10:55:53 (18.3 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [ ]:
import gzip

In [ ]:
!gzip -d cc.en.300.vec.gz

In [ ]:
path_to_fasttext_file = './cc.en.300.vec'

embeddings_index = {}
with open(path_to_fasttext_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 2000000 word vectors.


# 1 - 4 Vectorizer parameters

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embedding_dim = 300 # Dimension of embedded representation 
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 128 # Output dimension after vectorizing

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 28300 words (1700 misses)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = keras.layers.Dropout(0.6)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.6)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 128)              0         
 ectorization)                                                   
                                                                 
 embedding_17 (Embedding)    (None, 128, 300)          9000600   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128, 128)         186880    
 nal)                                                            
                                                                 
 dropout_36 (Dropout)        (None, 128, 128)          0         
                                                                 
 flatten_17 (Flatten)        (None, 16384)             0  

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 14
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/14
12/12 [==============================] - 8s 521ms/step - loss: 0.6921 - accuracy: 0.5199 - val_loss: 0.6845 - val_accuracy: 0.5240
Epoch 2/14
12/12 [==============================] - 7s 575ms/step - loss: 0.6774 - accuracy: 0.5702 - val_loss: 0.6680 - val_accuracy: 0.5720
Epoch 3/14
12/12 [==============================] - 6s 477ms/step - loss: 0.6554 - accuracy: 0.6153 - val_loss: 0.6364 - val_accuracy: 0.6320
Epoch 4/14
12/12 [==============================] - 6s 480ms/step - loss: 0.6129 - accuracy: 0.6647 - val_loss: 0.5890 - val_accuracy: 0.6970
Epoch 5/14
12/12 [==============================] - 6s 473ms/step - loss: 198.2994 - accuracy: 0.7261 - val_loss: 0.5567 - val_accuracy: 0.7170
Epoch 6/14
12/12 [==============================] - 6s 486ms/step - loss: 0.5297 - accuracy: 0.7386 - val_loss: 0.5279 - val_accuracy: 0.7440
Epoch 7/14
12/12 [==============================] - 6s 475ms/step - loss: 87.3008 - accuracy: 0.7453 - val_loss: 0.8457 - val_accuracy: 0.5660
Epo

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 78.79% 
f1-score is 0.8091424457842168% 
              precision    recall  f1-score   support

           0       0.87      0.68      0.76      4984
           1       0.74      0.90      0.81      5016

    accuracy                           0.79     10000
   macro avg       0.80      0.79      0.79     10000
weighted avg       0.80      0.79      0.79     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = keras.layers.Dropout(0.6)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.6)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 128)              0         
 ectorization)                                                   
                                                                 
 embedding_18 (Embedding)    (None, 128, 300)          9000600   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128, 128)         186880    
 nal)                                                            
                                                                 
 dropout_38 (Dropout)        (None, 128, 128)          0         
                                                                 
 flatten_18 (Flatten)        (None, 16384)             0  

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 14
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/14
12/12 [==============================] - 6s 484ms/step - loss: 0.5559 - accuracy: 0.7184 - val_loss: 0.5542 - val_accuracy: 0.7210
Epoch 2/14
12/12 [==============================] - 6s 471ms/step - loss: 0.5521 - accuracy: 0.7201 - val_loss: 0.5531 - val_accuracy: 0.7210
Epoch 3/14
12/12 [==============================] - 6s 467ms/step - loss: 0.5516 - accuracy: 0.7258 - val_loss: 0.5522 - val_accuracy: 0.7230
Epoch 4/14
12/12 [==============================] - 6s 473ms/step - loss: 0.5485 - accuracy: 0.7311 - val_loss: 0.5514 - val_accuracy: 0.7240
Epoch 5/14
12/12 [==============================] - 6s 482ms/step - loss: 0.5502 - accuracy: 0.7249 - val_loss: 0.5503 - val_accuracy: 0.7220
Epoch 6/14
12/12 [==============================] - 6s 460ms/step - loss: 0.5486 - accuracy: 0.7248 - val_loss: 0.5495 - val_accuracy: 0.7230
Epoch 7/14
12/12 [==============================] - 6s 475ms/step - loss: 0.5482 - accuracy: 0.7260 - val_loss: 0.5489 - val_accuracy: 0.7230
Epoch 

In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 12s 40ms/step - loss: 0.5521 - accuracy: 0.7289


[0.5521315336227417, 0.7289000153541565]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 72.89% 
f1-score is 0.726078609679701% 
              precision    recall  f1-score   support

           0       0.72      0.74      0.73      4984
           1       0.74      0.72      0.73      5016

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(128, activation='relu', return_sequences=True)(emb)
x = keras.layers.Dropout(0.2)(x)
x = GRU(128, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 128)              0         
 ectorization)                                                   
                                                                 
 embedding_20 (Embedding)    (None, 128, 300)          9000600   
                                                                 
 lstm_21 (LSTM)              (None, 128, 128)          219648    
                                                                 
 dropout_41 (Dropout)        (None, 128, 128)          0         
                                                                 
 gru_10 (GRU)                (None, 128, 128)          99072     
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 8
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/8
30/30 [==============================] - 19s 551ms/step - loss: 0.6766 - accuracy: 0.5671 - val_loss: 0.6312 - val_accuracy: 0.6680
Epoch 2/8
30/30 [==============================] - 17s 580ms/step - loss: 0.5585 - accuracy: 0.7216 - val_loss: 0.4711 - val_accuracy: 0.7832
Epoch 3/8
30/30 [==============================] - 27s 881ms/step - loss: 0.4630 - accuracy: 0.7867 - val_loss: 0.4223 - val_accuracy: 0.8100
Epoch 4/8
30/30 [==============================] - 16s 528ms/step - loss: 0.4223 - accuracy: 0.8115 - val_loss: 0.3923 - val_accuracy: 0.8288
Epoch 5/8
30/30 [==============================] - 16s 530ms/step - loss: 0.4025 - accuracy: 0.8232 - val_loss: 0.3857 - val_accuracy: 0.8264
Epoch 6/8
30/30 [==============================] - 16s 521ms/step - loss: 0.3784 - accuracy: 0.8349 - val_loss: 0.3705 - val_accuracy: 0.8408
Epoch 7/8
30/30 [==============================] - 16s 531ms/step - loss: 0.3714 - accuracy: 0.8376 - val_loss: 0.3645 - val_accuracy: 0.8540
Epoch 

In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 16s 51ms/step - loss: 0.3583 - accuracy: 0.8447


[0.35831525921821594, 0.8446999788284302]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 84.47% 
f1-score is 0.8460089241447694% 
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      4984
           1       0.84      0.85      0.85      5016

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = keras.layers.Dropout(0.2)(x)
x = Bidirectional(GRU(128, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 128)              0         
 ectorization)                                                   
                                                                 
 embedding_26 (Embedding)    (None, 128, 300)          9000600   
                                                                 
 bidirectional_13 (Bidirecti  (None, 128, 256)         439296    
 onal)                                                           
                                                                 
 dropout_65 (Dropout)        (None, 128, 256)          0         
                                                                 
 bidirectional_14 (Bidirecti  (None, 128, 256)         296

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 6
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/6
30/30 [==============================] - 77s 1s/step - loss: 0.6886 - accuracy: 0.5378 - val_loss: 0.6594 - val_accuracy: 0.6060
Epoch 2/6
30/30 [==============================] - 30s 1s/step - loss: 0.5741 - accuracy: 0.7091 - val_loss: 0.5363 - val_accuracy: 0.7432
Epoch 3/6
30/30 [==============================] - 31s 1s/step - loss: 0.4661 - accuracy: 0.7880 - val_loss: 0.4549 - val_accuracy: 0.7856
Epoch 4/6
30/30 [==============================] - 30s 999ms/step - loss: 0.4222 - accuracy: 0.8132 - val_loss: 0.4087 - val_accuracy: 0.8112
Epoch 5/6
30/30 [==============================] - 30s 1s/step - loss: 0.3924 - accuracy: 0.8309 - val_loss: 0.3708 - val_accuracy: 0.8372
Epoch 6/6
30/30 [==============================] - 30s 997ms/step - loss: 0.3651 - accuracy: 0.8439 - val_loss: 0.4000 - val_accuracy: 0.8220


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 26s 82ms/step - loss: 0.3653 - accuracy: 0.8405


[0.3652898073196411, 0.840499997138977]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

# 5 - 6 Vectorizer parameters

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embedding_dim = 300 # Dimension of embedded representation
vocab_size = 100000 # Number of unique tokens in vocabulary
sequence_length = 250 # Output dimension after vectorizing 

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 59576 words (14907 misses)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 5.Experiment

In [ ]:
#4
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
#x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_9 (TextV  (None, 300)              0         
 ectorization)                                                   
                                                                 
 embedding_18 (Embedding)    (None, 300, 300)          22345500  
                                                                 
 bidirectional_27 (Bidirecti  (None, 300, 128)         186880    
 onal)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 38400)             0         
                                                                 
 dense_53 (Dense)            (None, 128)               491

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 6
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = keras.layers.Dropout(0.2)(x)
x = Bidirectional(LSTM(128))(x)
x = keras.layers.Dense(128, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_4 (TextV  (None, 250)              0         
 ectorization)                                                   
                                                                 
 embedding_28 (Embedding)    (None, 250, 300)          22345500  
                                                                 
 bidirectional_17 (Bidirecti  (None, 250, 256)         439296    
 onal)                                                           
                                                                 
 dropout_73 (Dropout)        (None, 250, 256)          0         
                                                                 
 bidirectional_18 (Bidirecti  (None, 256)              394

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 6
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/6
30/30 [==============================] - 40s 1s/step - loss: 0.6837 - accuracy: 0.5531 - val_loss: 0.6334 - val_accuracy: 0.6480
Epoch 2/6
30/30 [==============================] - 32s 1s/step - loss: 0.5955 - accuracy: 0.6977 - val_loss: 0.4962 - val_accuracy: 0.7676
Epoch 3/6
30/30 [==============================] - 33s 1s/step - loss: 0.5250 - accuracy: 0.7603 - val_loss: 0.4817 - val_accuracy: 0.7848
Epoch 4/6
30/30 [==============================] - 32s 1s/step - loss: 0.4447 - accuracy: 0.8082 - val_loss: 0.4048 - val_accuracy: 0.8268
Epoch 5/6
30/30 [==============================] - 33s 1s/step - loss: 0.3903 - accuracy: 0.8362 - val_loss: 0.3753 - val_accuracy: 0.8384
Epoch 6/6
30/30 [==============================] - 33s 1s/step - loss: 0.3625 - accuracy: 0.8495 - val_loss: 0.3552 - val_accuracy: 0.8488


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 28s 89ms/step - loss: 0.3599 - accuracy: 0.8489


[0.3598681688308716, 0.8489000201225281]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 84.89% 
f1-score is 0.8502626102467544% 
              precision    recall  f1-score   support

           0       0.85      0.84      0.85      4984
           1       0.85      0.86      0.85      5016

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



# Save model

In [ ]:
model.save('IMDB_FT_fourth')

INFO:tensorflow:Assets written to: FR_LSTM_sixth/assets


In [ ]:
!zip -r /content/IMDB_FT_first.zip /content/IMDB_FT_first/

  adding: content/FR_LSTM_sixth/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/variables.index (deflated 67%)
  adding: content/FR_LSTM_sixth/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/FR_LSTM_sixth/assets/ (stored 0%)
  adding: content/FR_LSTM_sixth/keras_metadata.pb (deflated 90%)
  adding: content/FR_LSTM_sixth/saved_model.pb (deflated 78%)
